In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

### Data Collection

In [ ]:
import requests

# URLs of the files
train_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module5/exercise/module5_exercise_train.csv'
test_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module5/exercise/module5_exercise_test.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

# Downloading the files
download_file(train_data_url, 'module5_exercise_train.csv')
download_file(test_data_url, 'module5_exercise_test.csv')

In [ ]:
df_train =  pd.read_csv("module5_exercise_train.csv", sep=",")
df_test =  pd.read_csv("module5_exercise_test.csv", sep=",")

### Data analysis

In [ ]:
#### Make a complete analysis on data preprocessing
# Inconsistencies
# Duplicates (data.duplicated().sum())
# Missing values (data.isnull().sum())
# Categorical
# Outliers
# Feature Engineering
# Feature Selection and/or Dimensionality Reduction

In [ ]:
data_df = pd.concat([df_train, df_test], axis=0)
data_df.head(5)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
def plot_feature_over_time(df, feature, date_id_start, date_id_end):
    df_filtered = df[(df['date'] >= date_id_start) & (df['date'] <= date_id_end)]
    
    if feature not in df_filtered.columns:
        print(f"Feature '{feature}' not found in the DataFrame.")
        return
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(df_filtered['date'], df_filtered[feature], label=feature, linestyle='-')
    plt.xlabel('Date')
    plt.ylabel(feature)
    plt.title(f'{feature} from {date_id_start} to {date_id_end}')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()



In [ ]:
data_df['date'] = pd.to_datetime(data_df['date'])

In [ ]:
data_df.head(5)

In [ ]:
data_df['wind_speed']

In [ ]:
plot_feature_over_time(data_df, 'electricity_demand', '2017-09-01', '2017-10-01')

In [ ]:
plot_feature_over_time(data_df, 'temperature_station1', '2017-09-01', '2019-10-01')

In [ ]:
plot_feature_over_time(data_df, 'humidity', '2016-06-01', '2016-12-01')

### Data analysis by Guillaume

In [ ]:
data_df.head(5)

In [ ]:
data_df.sample(5)

#### Dates format

In [ ]:
import re
# print(data_df['date'].apply(lambda x: not re.match(r'\d{4}-\d{2}-\d{2}', x)).sum())
test = pd.to_datetime(data_df['date'])
# data_df['date'] = pd.to_datetime(data_df['date'])
test

There is no probleme with date format : no need to make changes.
There is a problem with speed, we need to either have all values in m/s or km/h.

#### Speed unit

In [ ]:
def convert_wind_speed(string):
    if isinstance(string, (int, float)):
        return string
    elif pd.isna(string):
        return string
    elif string[-1] == 'h':
        return float(string.replace('km/h', '')) * 3.6
    else:
        return float(string.replace('m/s', ''))
    
print(convert_wind_speed('100 km/h'))
print(convert_wind_speed('100 m/s'))

test = data_df.copy()
test['wind_speed'] = data_df['wind_speed'].apply(convert_wind_speed)
test.head(5)

#### Duplicates

In [ ]:
test = data_df.copy()

In [ ]:
print(test.shape)
test.drop_duplicates(inplace=True)
print(test.shape)

#### Categorical values

In [ ]:
data_df.head(2)

In [ ]:
df = data_df.copy()
categorical_columns = ['weather_condition', 'oil_brent_price_indicator']
df = df[categorical_columns]
print(df['weather_condition'].value_counts())
df['oil_brent_price_indicator'].value_counts()

In [ ]:

df = data_df.copy()
categorical_columns = ['weather_condition', 'oil_brent_price_indicator']
df = df[categorical_columns]
print("Original DataFrame:")
print(df)

# One-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_columns)

print("One-hot encoded DataFrame:")
print(df_encoded)          

In [ ]:
df = data_df.copy()

# Diagnose
print("Duplicate index?", df.index.has_duplicates)
print("Duplicate columns?", df.columns.duplicated().any())
print(df.columns[df.columns.duplicated()])

# Fixes
df = df.reset_index(drop=True)                     # handles duplicate index
df = df.loc[:, ~df.columns.duplicated()]          # drops duplicate-named columns, if any


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# df = data_df.drop_duplicates().copy()

# Count plot for 'weather_condition'
plt.figure(figsize=(10, 6))
sns.countplot(x='weather_condition', data=df, palette='viridis')
plt.title('Count Plot of weather_condition')
plt.xlabel('weather_condition')
plt.ylabel('Count')
plt.show()

# Bar plot for average rating by color
plt.figure(figsize=(10, 6))
sns.barplot(x='weather_condition', y='electricity_demand', data=df, ci=None, palette='viridis')
plt.title('Average electricity_demand by weather_condition')
plt.xlabel('weather_condition')
plt.ylabel('Average electricity_demand')
plt.show()

# Stacked bar plot for size distribution by color
size_color = pd.crosstab(df['weather_condition'], df['oil_brent_price_indicator'])
size_color_pct = size_color.div(size_color.sum(1), axis=0)
size_color_pct.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('oil_brent_price_indicator Distribution by weather_condition')
plt.xlabel('weather_condition')
plt.ylabel('Percentage')
plt.legend(title='oil_brent_price_indicator', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
            

### Data Preprocessing Evaluation Strategy

In [ ]:
# Provide a complete data preprocessing transformations

#### Preprocessing functions

In [ ]:
from sklearn.preprocessing import LabelEncoder


# 1. Handle Inconsistencies
def handle_inconsistencies(X_train, y_train, X_val=None):
    if X_val is not None:
        X_train['wind_speed'] = X_train['wind_speed'].apply(convert_wind_speed)
        X_val['wind_speed'] = X_val['wind_speed'].apply(convert_wind_speed)
        return X_train.copy(), y_train, X_val.copy()
    else:
        X_train['wind_speed'] = X_train['wind_speed'].apply(convert_wind_speed)
        return X_train.copy()

# 2. Handling Duplicates
def handle_duplicates(X_train, y_train, X_val=None):
    if X_val is not None:
        X_train_no_duplicates = X_train.drop_duplicates().copy()
        y_train_no_duplicates = y_train.loc[X_train_no_duplicates.index]
        X_val_no_duplicates = X_val.drop_duplicates().copy()
        return X_train_no_duplicates, y_train_no_duplicates, X_val_no_duplicates
    else:
        X_train_no_duplicates = X_train.copy()
        y_train_no_duplicates = y_train.loc[X_train_no_duplicates.index]
        return X_train_no_duplicates, y_train_no_duplicates


def handle_missing_values(X_train, y_train=None, X_val=None):
    # Colonnes numériques et catégorielles
    num_cols = X_train.select_dtypes(include=["number"]).columns
    cat_cols = X_train.select_dtypes(include=["object", "category"]).columns

    # Remplissage sur X_train
    X_train[num_cols] = X_train[num_cols].fillna(-1)
    X_train[cat_cols] = X_train[cat_cols].fillna("missing")

    if X_val is not None:
        X_val[num_cols] = X_val[num_cols].fillna(-1)
        X_val[cat_cols] = X_val[cat_cols].fillna("missing")
        return X_train.copy(), y_train.copy(), X_val.copy()
    else:
        return X_train.copy()


# 4. Handling Categorical Values
def handle_categorical(X_train, y_train, X_val=None):
    cat_cols = X_train.select_dtypes(include=['object', 'category']).columns
    cat_cols = cat_cols.drop('date')
    if X_val is not None:
        for col in cat_cols:
            labelencoder = LabelEncoder()
            X_train[col] = labelencoder.fit_transform(X_train[col])
            X_val[col] = labelencoder.transform(X_val[col])
        return X_train.copy(), y_train.copy(), X_val.copy()
    else:
        for col in cat_cols:
            labelencoder = LabelEncoder()
            X_train[col] = labelencoder.fit_transform(X_train[col])
        return X_train.copy(), y_train.copy()

# 5. Handling Outliers
def handle_outliers(X_train, y_train, X_val=None):
    if X_val is not None:
        return X_train.copy(), y_train, X_val.copy()
    else:
        return X_train.copy(), y_train

# 6. Feature Engineering
def feature_engineering(X_train, y_train, X_val=None):
    if X_val is not None:
        return X_train.copy(), X_val.copy()
    else:
        return X_train.copy()

# 7. Feature Selection and Dimensionality Reduction
def feature_selection(X_train, y_train, X_val=None):
    selected_columns = ['weather_condition', 'humidity', 'wind_speed', 'oil_brent_price_indicator', 'temperature_station1',
       'temperature_station2', 'temperature_station3', 'temperature_station4',
       'temperature_station5', 'temperature_station6', 'temperature_station7',
       'temperature_station8', 'temperature_station9', 'temperature_station10']
    if X_val is not None:
        return X_train[selected_columns], X_val[selected_columns]
    else:
        return X_train[selected_columns]

#### Pipeline function

In [ ]:
def evaluate_pipeline(X, y, n_splits=5):

    # ### call transformations here, if there is no learning and no need to be crossval
    # X = handle_inconsistencies(X, y)
    # X, y = handle_duplicates(X, y)
    # X = handle_missing_values(X, y)
    # X, y = handle_categorical(X, y)
    # # X, y = handle_outliers(X, y)
    # # X, y = feature_engineering(XX, y)
    # X = feature_selection(X, y)
    
    X = X.copy().sort_values('date')
    y = y.copy().reindex(X.index).reset_index(drop=True)
    X = X.reset_index(drop=True)
    model = LinearRegression()
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    train_scores = []
    val_scores = []


    for fold, (train_index, val_index) in enumerate(tscv.split(X)):
        print(f"Processing fold {fold + 1}/{n_splits}...")
        
        # Split data into train and validation sets
        X_train, X_val = X.iloc[train_index].copy(), X.iloc[val_index].copy()
        y_train, y_val = y.iloc[train_index].copy(), y.iloc[val_index].copy()

        ### call transformations here, if there is learning
        X_train, y_train, X_val = handle_inconsistencies(X_train, y_train, X_val)
        X_train, y_train, X_val= handle_duplicates(X_train, y_train, X_val)
        y_val = y_val.loc[X_val.index]
        X_train, y_train, X_val = handle_missing_values(X_train, y_train, X_val)
        X_train, y_train, X_val = handle_categorical(X_train, y_train, X_val)
        X_train, y_train, X_val = handle_outliers(X_train, y_train, X_val)
        y_val = y_val.loc[X_val.index]
        X_train, X_val = feature_engineering(X_train, y_train, X_val)
        X_train, X_val = feature_selection(X_train, y_train, X_val)
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on training set
        y_train_pred = model.predict(X_train)
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_scores.append(train_mse)
        
        # Predict on validation set
        y_val_pred = model.predict(X_val)
        val_mse = mean_squared_error(y_val, y_val_pred)
        val_scores.append(val_mse)
        
        print(f"Fold {fold + 1} Train MSE: {train_mse:.4f}, Validation MSE: {val_mse:.4f}")
    
    # Compute mean, max, and min values for train and validation MSE
    mean_train_mse = np.mean(train_scores)
    max_train_mse = np.max(train_scores)
    min_train_mse = np.min(train_scores)
    
    mean_val_mse = np.mean(val_scores)
    max_val_mse = np.max(val_scores)
    min_val_mse = np.min(val_scores)
    
    # Print results
    print("\nTrain MSE:")
    print(f"Mean: {mean_train_mse:.4f}, Max: {max_train_mse:.4f}, Min: {min_train_mse:.4f}")
    
    print("\nValidation MSE:")
    print(f"Mean: {mean_val_mse:.4f}, Max: {max_val_mse:.4f}, Min: {min_val_mse:.4f}")
    
    return mean_val_mse  # Return mean validation MSE as the overall score

#### First try : simple case

In [ ]:
# Prepare X and y
X = df_train.copy().drop(columns=['electricity_demand'], axis=1)
y = df_train.copy().pop('electricity_demand')

# Run the evaluation
evaluate_pipeline(X, y)

#### Second try : handling outliers 

Going to try to handle odd values 

In [ ]:
df = data_df.copy()
df.head(2)

In [ ]:
import numpy as np
from scipy import stats

def detect_outliers_zscore(data, threshold=5):
    z_scores = np.abs(stats.zscore(data, nan_policy='omit'))
    return data[z_scores > threshold]

df1 = df.select_dtypes(include=[np.number])
for col in df1.columns:
    outliers = detect_outliers_zscore(df[col])
    print(outliers)
                

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20, 15))  
df = df.reset_index(drop=True)
for i, col in enumerate(df1.columns, start=1):
    # plt.figure(figsize=(10, 6))
    plt.subplot(5, 6, i)
    sns.boxplot(x=col, data=df)
    plt.title(f'{col}')
plt.tight_layout()
plt.show()
                

In [ ]:
# # 5. New Handling Outliers
# def handle_outliers(X_train, y_train, X_val=None, y_val=None):
#     df_num = X_train.select_dtypes(include=[np.number])    # Sélection des colonnes numériques
#     outlier_indices_train = set()
#     outlier_indices_val = set()
#     if X_val is not None:
#         for col in df_num.columns:
#             outliers_train = detect_outliers_zscore(X_train[col])
#             outliers_val = detect_outliers_zscore(X_val[col])
#             if not outliers_train.empty or not outliers_val.empty:
#                 outlier_indices_train.update(outliers_train.index)
#                 outlier_indices_val.update(outliers_val.index)
#         X_train, X_val = X_train.drop(index=outlier_indices_train), X_val.drop(index=outlier_indices_val)
#         y_train, y_val = y_train.loc[X_train.index], y_val.loc[X_val.index]
#         return X_train.copy(), y_train, X_val.copy(), y_val.copy()
#     else:
#         for col in df_num.columns:
#             outliers_train = detect_outliers_zscore(X_train[col])
#             if not outliers_train.empty:
#                 outlier_indices_train.update(outliers_train.index)
#         X_train = X_train.drop(index=outlier_indices_train)
#         y_train = y_train.loc[X_train.index]
#         return X_train.copy(), y_train

In [ ]:
def handle_outliers(X_train, y_train, X_val=None):
    df_num = X_train.select_dtypes(include=[np.number])
    outlier_indices_train, outlier_indices_val = set(), set()

    if X_val is not None:
        for col in df_num.columns:
            outliers_X_train = detect_outliers_zscore(X_train[col])
            outliers_X_val = detect_outliers_zscore(X_val[col])
            if not outliers_X_train.empty: outlier_indices_train.update(outliers_X_train.index)
            if not outliers_X_val.empty: outlier_indices_val.update(outliers_X_val.index)

        outliers_y_train = detect_outliers_zscore(y_train)
        if not outliers_y_train.empty: outlier_indices_train.update(outliers_y_train.index)

        X_train = X_train.drop(index=outlier_indices_train)
        X_val   = X_val.drop(index=outlier_indices_val)
        y_train = y_train.loc[X_train.index]
        return X_train.copy(), y_train, X_val.copy()

    else:
        for col in df_num.columns:
            outliers_X_train = detect_outliers_zscore(X_train[col])
            if not outliers_X_train.empty: outlier_indices_train.update(outliers_X_train.index)

        outliers_y_train = detect_outliers_zscore(y_train)
        if not outliers_y_train.empty: outlier_indices_train.update(outliers_y_train.index)

        X_train = X_train.drop(index=outlier_indices_train)
        y_train = y_train.loc[X_train.index]
        return X_train.copy(), y_train


In [ ]:
# Prepare X and y
X = df_train.copy().drop(columns=['electricity_demand'], axis=1)
y = df_train.copy().pop('electricity_demand')

# Run the evaluation
evaluate_pipeline(X, y)

Processing fold 1/5...
Fold 1 Train MSE: 1208.3526, Validation MSE: 1525.8924
Processing fold 2/5...
Fold 2 Train MSE: 1340.3060, Validation MSE: 1383.4770
Processing fold 3/5...
Fold 3 Train MSE: 1303.0857, Validation MSE: 164593635.8174
Processing fold 4/5...
Fold 4 Train MSE: 40840900.6609, Validation MSE: 229449.1393
Processing fold 5/5...
Fold 5 Train MSE: 32804080.7004, Validation MSE: 119774.5234

Train MSE:
Mean: 14729766.6211, Max: 40840900.6609, Min: 1208.3526

Validation MSE:
Mean: 32989153.7699, Max: 164593635.8174, Min: 1383.4770
32989153.76988711
NO CHANGE !!

#### Third try : Correlation

In [ ]:
df = data_df.copy()
corr_matrix = df.corr(numeric_only=True)  # numeric_only=True pour éviter les erreurs avec les colonnes non numériques

plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Matrice de corrélation")
plt.show()

Toutes les temperature_station* sont corrélées : soit on en garde qu'une, soit on fait une moyenne. 
On remarque que temperature_station7 est mieux corrélée que les autrs à y. On va voir ce qu'il se trame

In [ ]:
plot_feature_over_time(data_df, 'temperature_station7', '2017-09-01', '2019-10-01')

Bah temperature_station7 fait n'importe quoi en gros. On va donc la drop, puis faire une moyenne des autres colonnes. 

In [ ]:
df = data_df.copy()
df.head(2)

In [ ]:
# 6. Feature Engineering
def feature_engineering(X_train, y_train, X_val=None):
    temp_cols = ['temperature_station1', 'temperature_station2', 'temperature_station3', 'temperature_station4', 'temperature_station5', 'temperature_station6', 'temperature_station8', 'temperature_station9', 'temperature_station10']
    if X_val is not None:
        X_train, X_val = X_train.drop('temperature_station7', axis=1), X_val.drop('temperature_station7', axis=1)
        X_train['temperature_mean'], X_val['temperature_mean'] = X_train[temp_cols].mean(axis=1), X_val[temp_cols].mean(axis=1)
        X_train, X_val = X_train.drop(columns=temp_cols), X_val.drop(columns=temp_cols)
        return X_train.copy(), X_val.copy()
    else:
        X_train = X_train.drop('temperature_station7', axis=1)
        X_train['temperature_mean'] = X_train[temp_cols].mean(axis=1)
        X_train= X_train.drop(columns=temp_cols)
        return X_train.copy()
    
# 7. Feature Selection and Dimensionality Reduction
def feature_selection(X_train, y_train, X_val=None):
    selected_columns = ['weather_condition', 'humidity', 'wind_speed', 'oil_brent_price_indicator', 'temperature_mean']
    if X_val is not None:
        return X_train[selected_columns], X_val[selected_columns]
    else:
        return X_train[selected_columns]

In [ ]:
def handle_outliers(X_train, y_train, X_val=None):
    df_num = X_train.select_dtypes(include=[np.number])
    outlier_indices_train, outlier_indices_val = set(), set()

    if X_val is not None:
        for col in df_num.columns:
            outliers_X_train = detect_outliers_zscore(X_train[col])
            outliers_X_val = detect_outliers_zscore(X_val[col])
            if not outliers_X_train.empty: outlier_indices_train.update(outliers_X_train.index)
            if not outliers_X_val.empty: outlier_indices_val.update(outliers_X_val.index)

        outliers_y_train = detect_outliers_zscore(y_train)
        if not outliers_y_train.empty: outlier_indices_train.update(outliers_y_train.index)

        X_train = X_train.drop(index=outlier_indices_train)
        X_val   = X_val.drop(index=outlier_indices_val)
        y_train = y_train.loc[X_train.index]
        return X_train.copy(), y_train, X_val.copy()

    else:
        for col in df_num.columns:
            outliers_X_train = detect_outliers_zscore(X_train[col])
            if not outliers_X_train.empty: outlier_indices_train.update(outliers_X_train.index)

        outliers_y_train = detect_outliers_zscore(y_train)
        if not outliers_y_train.empty: outlier_indices_train.update(outliers_y_train.index)

        X_train = X_train.drop(index=outlier_indices_train)
        y_train = y_train.loc[X_train.index]
        return X_train.copy(), y_train


In [ ]:
# Prepare X and y
X = df_train.copy().drop(columns=['electricity_demand'], axis=1)
y = df_train.copy().pop('electricity_demand')

# Run the evaluation
evaluate_pipeline(X, y)

In [ ]:
# Processing fold 1/5...
# Fold 1 Train MSE: 1208.3526, Validation MSE: 1525.8924
# Processing fold 2/5...
# Fold 2 Train MSE: 1340.3060, Validation MSE: 1383.4770
# Processing fold 3/5...
# Fold 3 Train MSE: 1303.0857, Validation MSE: 164914287.1979
# Processing fold 4/5...
# Fold 4 Train MSE: 40870901.0860, Validation MSE: 234699.9561
# Processing fold 5/5...
# Fold 5 Train MSE: 32845232.1681, Validation MSE: 123151.9030

# Train MSE:
# Mean: 14743996.9997, Max: 40870901.0860, Min: 1208.3526

# Validation MSE:
# Mean: 33055009.6853, Max: 164914287.1979, Min: 1383.4770

#### Fourth try : handle better missing values

In [ ]:
plot_feature_over_time(data_df, 'temperature_station1', '2017-09-01', '2019-10-01')

In [ ]:
df = data_df.drop_duplicates().copy()
df.isna().sum()
df = df.ffill()

In [ ]:
plot_feature_over_time(df, 'temperature_station1', '2017-09-01', '2019-10-01')

In [ ]:
data_df.plot(y='date', use_index=True)

In [ ]:
data_df.sort_values('date').reset_index(drop=True).plot(y='date', use_index=True)

In [ ]:
df = data_df.sort_values('date').reset_index(drop=True).copy()
df = df.ffill()
plot_feature_over_time(df, 'temperature_station1', '2017-09-01', '2019-10-01')

In [ ]:
df1 = data_df.copy()
df1 = df1.ffill().bfill()
print(df1.isna().sum())

In [ ]:
df = data_df.sort_values('date').reset_index(drop=True).copy()
df = df.fillna(method='ffill')
print(df.isna().sum())
plot_feature_over_time(df, 'temperature_station2', '2017-09-01', '2019-10-01')

In [ ]:
def handle_missing_values(X_train, y_train, X_val=None):
    if X_val is not None:
        X_train, X_val = X_train.sort_values('date'), X_val.sort_values('date')
        y_train = y_train.reindex(X_train.index).reset_index(drop=True)
        X_train, X_val = X_train.reset_index(drop=True).ffill().bfill(), X_val.ffill().bfill()
        return X_train.copy(), y_train.copy(), X_val.copy()
    else:
        X_train = X_train.sort_values('date')
        y_train = y_train.reindex(X_train.index).reset_index(drop=True)
        X_train = X_train.reset_index(drop=True).ffill().bfill()
        return X_train.copy(), y_train.copy()
    

In [ ]:
# 6. Feature Engineering
def feature_engineering(X_train, y_train, X_val=None):
    temp_cols = ['temperature_station1', 'temperature_station2', 'temperature_station3', 'temperature_station4', 'temperature_station5', 'temperature_station6', 'temperature_station8', 'temperature_station9', 'temperature_station10']
    if X_val is not None:
        X_train, X_val = X_train.drop('temperature_station7', axis=1), X_val.drop('temperature_station7', axis=1)
        X_train['temperature_mean'], X_val['temperature_mean'] = X_train[temp_cols].mean(axis=1), X_val[temp_cols].mean(axis=1)
        X_train, X_val = X_train.drop(columns=temp_cols), X_val.drop(columns=temp_cols)
        return X_train.copy(), X_val.copy()
    else:
        X_train = X_train.drop('temperature_station7', axis=1)
        X_train['temperature_mean'] = X_train[temp_cols].mean(axis=1)
        X_train= X_train.drop(columns=temp_cols)
        return X_train.copy()
    
# 7. Feature Selection and Dimensionality Reduction
def feature_selection(X_train, y_train, X_val=None):
    selected_columns = ['weather_condition', 'humidity', 'wind_speed', 'oil_brent_price_indicator', 'temperature_mean']
    if X_val is not None:
        return X_train[selected_columns], X_val[selected_columns]
    else:
        return X_train[selected_columns]

In [ ]:
def handle_outliers(X_train, y_train, X_val=None):
    df_num = X_train.select_dtypes(include=[np.number])
    outlier_indices_train, outlier_indices_val = set(), set()

    if X_val is not None:
        for col in df_num.columns:
            outliers_X_train = detect_outliers_zscore(X_train[col])
            outliers_X_val = detect_outliers_zscore(X_val[col])
            if not outliers_X_train.empty: outlier_indices_train.update(outliers_X_train.index)
            if not outliers_X_val.empty: outlier_indices_val.update(outliers_X_val.index)

        outliers_y_train = detect_outliers_zscore(y_train)
        if not outliers_y_train.empty: outlier_indices_train.update(outliers_y_train.index)

        X_train = X_train.drop(index=outlier_indices_train)
        X_val   = X_val.drop(index=outlier_indices_val)
        y_train = y_train.loc[X_train.index]
        return X_train.copy(), y_train, X_val.copy()

    else:
        for col in df_num.columns:
            outliers_X_train = detect_outliers_zscore(X_train[col])
            if not outliers_X_train.empty: outlier_indices_train.update(outliers_X_train.index)

        outliers_y_train = detect_outliers_zscore(y_train)
        if not outliers_y_train.empty: outlier_indices_train.update(outliers_y_train.index)

        X_train = X_train.drop(index=outlier_indices_train)
        y_train = y_train.loc[X_train.index]
        return X_train.copy(), y_train



In [ ]:
# Prepare X and y
X = df_train.copy().drop(columns=['electricity_demand'], axis=1)
y = df_train.copy().pop('electricity_demand')

# Run the evaluation
evaluate_pipeline(X, y)

In [ ]:
# Processing fold 1/5...
# Fold 1 Train MSE: 1229.5997, Validation MSE: 1502.3436
# Processing fold 2/5...
# Fold 2 Train MSE: 1356.9838, Validation MSE: 1355.6208
# Processing fold 3/5...
# Fold 3 Train MSE: 1315.7972, Validation MSE: 1494.4123
# Processing fold 4/5...
# Fold 4 Train MSE: 1357.5094, Validation MSE: 1407.0646
# Processing fold 5/5...
# Fold 5 Train MSE: 1364.8561, Validation MSE: 1248.3827

# Train MSE:
# Mean: 1324.9492, Max: 1364.8561, Min: 1229.5997

# Validation MSE:
# Mean: 1401.5648, Max: 1502.3436, Min: 1248.3827
# 1401.5648162556543

#### Fith try : Scaling and normalization

In [ ]:
def preprocessing_X_y(X, y):
    X = X.copy().sort_values('date')
    y = y.copy().reindex(X.index).reset_index(drop=True)
    X = X.reset_index(drop=True)    
    X = handle_inconsistencies(X, y)
    X, y = handle_duplicates(X, y)
    X, y = handle_missing_values(X, y)
    X, y = handle_categorical(X, y)
    X, y = handle_outliers(X, y,)
    X  = feature_engineering(X, y)
    X  = feature_selection(X, y)
    return X.copy(), y.copy()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, Normalizer, RobustScaler
# Prepare X and y
X = df_train.copy().drop(columns=['electricity_demand'], axis=1)
y = df_train.copy().pop('electricity_demand')

X, y = preprocessing_X_y(X, y)

# Function to train and evaluate
def train_and_evaluate(X_train, X_test, y_train, y_test):
    # model = KNeighborsRegressor(n_neighbors=5)
    model = LinearRegression()
    model.fit(X_train, y_train)
    mse_train = mean_squared_error(y_train, model.predict(X_train))
    mse_test = mean_squared_error(y_test, model.predict(X_test))
    return mse_train, mse_test


# Function to evaluate scaling strategy using cross-validation
def evaluate_scaling_strategy(X, y, scaler, name):

    tscv = TimeSeriesSplit(n_splits=5)
    # kf = KFold(n_splits=5, shuffle=True, random_state=42)
    train_mse_scores = []
    test_mse_scores = []
    
    # for train_index, test_index in kf.split(X):
    for fold, (train_index, test_index) in enumerate(tscv.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if scaler:
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
        else:
            X_train_scaled, X_test_scaled = X_train, X_test
        
        mse_train, mse_test = train_and_evaluate(X_train_scaled, X_test_scaled, y_train, y_test)
        
        train_mse_scores.append(mse_train)
        test_mse_scores.append(mse_test)
    
    return {
        'name': name,
        'train_mean': np.mean(train_mse_scores),
        'train_min': np.min(train_mse_scores),
        'train_max': np.max(train_mse_scores),
        'test_mean': np.mean(test_mse_scores),
        'test_min': np.min(test_mse_scores),
        'test_max': np.max(test_mse_scores)
    }

# Evaluate different scaling strategies
scaling_strategies = [
    (None, 'Original'),
    (MinMaxScaler(), 'Min-Max'),
    (StandardScaler(), 'Standard'),
    (MaxAbsScaler(), 'MaxAbs'),
    (Normalizer(norm='l2'), 'L2')
]

results = []
for scaler, name in scaling_strategies:
    result = evaluate_scaling_strategy(X, y, scaler, name)
    results.append(result)
    print(f"{name} Scaling - Train MSE: {result['train_mean']:.4f}, Test MSE: {result['test_mean']:.4f}")

In [ ]:
# Original Scaling - Train MSE: 395.2081, Test MSE: 599.0776
# Min-Max Scaling - Train MSE: 361.6628, Test MSE: 575.4965
# Standard Scaling - Train MSE: 354.4588, Test MSE: 561.9575
# MaxAbs Scaling - Train MSE: 317.1361, Test MSE: 505.4162
# L2 Scaling - Train MSE: 382.3350, Test MSE: 583.9914

In [ ]:
# Plot results
plt.figure(figsize=(12, 6))
x = range(len(results))
width = 0.35

plt.bar([i - width/2 for i in x], [r['train_mean'] for r in results], width, label='Train MSE', alpha=0.8)
plt.bar([i + width/2 for i in x], [r['test_mean'] for r in results], width, label='Test MSE', alpha=0.8)

for i, r in enumerate(results):
    plt.errorbar(i - width/2, r['train_mean'], yerr=[[r['train_mean']-r['train_min']], [r['train_max']-r['train_mean']]], fmt='none', capsize=5, color='black')
    plt.errorbar(i + width/2, r['test_mean'], yerr=[[r['test_mean']-r['test_min']], [r['test_max']-r['test_mean']]], fmt='none', capsize=5, color='black')

plt.xlabel('Scaling Strategy')
plt.ylabel('Mean Squared Error')
plt.title('Comparison of Scaling Strategies')
plt.xticks(x, [r['name'] for r in results])
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def normalize_standard_scaler(X_train, y_train, X_val=None):
    if X_val is not None:
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train).copy()
        X_val_scaled = scaler.transform(X_val).copy()
        return X_train_scaled, X_val_scaled
    else:
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train).copy()
        return X_train_scaled
    

def normalize(X_train, y_train, X_val=None, scaler=Normalizer(norm='l2')):
    if X_val is not None:
        X_train_scaled = scaler.fit_transform(X_train).copy()
        X_val_scaled = scaler.transform(X_val).copy()
        return X_train_scaled, X_val_scaled
    else:
        X_train_scaled = scaler.fit_transform(X_train).copy()
        return X_train_scaled
    
    

In [ ]:
def evaluate_pipeline(X, y, n_splits=5):

    ### Sort datasets by date
    X = X.copy().sort_values('date')
    y = y.copy().reindex(X.index).reset_index(drop=True)
    X = X.reset_index(drop=True)

    ### Deal with absurd data (electricity demand)
    q_low, q_high = y.quantile(0.01), y.quantile(0.99)
    mask = y.between(q_low, q_high)
    X, y = X[mask], y[mask]

    model = LinearRegression()
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    train_scores = []
    val_scores = []


    for fold, (train_index, val_index) in enumerate(tscv.split(X)):
        print(f"Processing fold {fold + 1}/{n_splits}...")
        
        # Split data into train and validation sets
        X_train, X_val = X.iloc[train_index].copy(), X.iloc[val_index].copy()
        y_train, y_val = y.iloc[train_index].copy(), y.iloc[val_index].copy()
        print(X_train.shape)

        ### call transformations here, if there is learning
        X_train, y_train, X_val = handle_inconsistencies(X_train, y_train, X_val)
        X_train, y_train, X_val= handle_duplicates(X_train, y_train, X_val)
        y_val = y_val.loc[X_val.index]
        X_train, y_train, X_val = handle_missing_values(X_train, y_train, X_val)
        X_train, y_train, X_val = handle_categorical(X_train, y_train, X_val)
        X_train, y_train, X_val = handle_outliers(X_train, y_train, X_val)
        y_val = y_val.loc[X_val.index]
        X_train, X_val = feature_engineering(X_train, y_train, X_val)
        X_train, X_val = feature_selection(X_train, y_train, X_val)
        # X_train, X_val = normalize_standard_scaler(X_train, y_train, X_val)
        X_train, X_val = normalize(X_train, y_train, X_val, scaler)
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on training set
        y_train_pred = model.predict(X_train)
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_scores.append(train_mse)
        
        # Predict on validation set
        y_val_pred = model.predict(X_val)
        val_mse = mean_squared_error(y_val, y_val_pred)
        val_scores.append(val_mse)
        
        print(f"Fold {fold + 1} Train MSE: {train_mse:.4f}, Validation MSE: {val_mse:.4f}")
    
    # Compute mean, max, and min values for train and validation MSE
    mean_train_mse = np.mean(train_scores)
    max_train_mse = np.max(train_scores)
    min_train_mse = np.min(train_scores)
    
    mean_val_mse = np.mean(val_scores)
    max_val_mse = np.max(val_scores)
    min_val_mse = np.min(val_scores)
    
    # Print results
    print("\nTrain MSE:")
    print(f"Mean: {mean_train_mse:.4f}, Max: {max_train_mse:.4f}, Min: {min_train_mse:.4f}")
    print("\nValidation MSE:")
    print(f"Mean: {mean_val_mse:.4f}, Max: {max_val_mse:.4f}, Min: {min_val_mse:.4f}")
    
    return mean_val_mse  # Return mean validation MSE as the overall score

In [ ]:
# Prepare X and y
X = df_train.copy().drop(columns=['electricity_demand'], axis=1)
y = df_train.copy().pop('electricity_demand')

# Run the evaluation
evaluate_pipeline(X, y)

#### Feature engineeringv2 : add interestings columns 

In [ ]:
df = data_df.copy()
def add_datetime_decomposition(df, datetime_col):
    df[datetime_col] = pd.to_datetime(df[datetime_col])
    df['year'] = df[datetime_col].dt.year
    df['month'] = df[datetime_col].dt.month
    df['day'] = df[datetime_col].dt.day
    df['hour'] = df[datetime_col].dt.hour
    df['dayofweek'] = df[datetime_col].dt.dayofweek
    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
    return df
df = add_datetime_decomposition(df, datetime_col='date')
df

In [ ]:
df = data_df.copy()
# Creating a 'temperature_feel' feature (heat index simplified formula)
def add_temperature_feel(df, temperature_col, humidity_col):
    df['temperature_feel'] = df[temperature_col] - 0.55 * (1 - df[humidity_col] / 100) * (df[temperature_col] - 14.5)
    return df

df = add_temperature_feel(df[['temperature_station1', 'humidity']], 'temperature_station1', 'humidity')
df.describe()

In [ ]:
df = data_df.copy()
# New transformation 2: Compute feels_like temperature
def compute_feels_like(temp, windspeed, humidity):
    feels_like = np.zeros_like(temp)
    
    # Heat index calculation (for high temperatures)
    mask_hi = temp > 27
    hi = -8.78469475556 + 1.61139411 * temp + 2.33854883889 * humidity
    hi += -0.14611605 * temp * humidity - 0.012308094 * temp**2
    hi += -0.0164248277778 * humidity**2 + 0.002211732 * temp**2 * humidity
    hi += 0.00072546 * temp * humidity**2 - 0.000003582 * temp**2 * humidity**2
    feels_like[mask_hi] = hi[mask_hi]
    
    # Wind chill calculation (for low temperatures with wind)
    mask_wc = (temp <= 10) & (windspeed > 4.8)
    wc = 13.12 + 0.6215 * temp - 11.37 * windspeed**0.16 + 0.3965 * temp * windspeed**0.16
    feels_like[mask_wc] = wc[mask_wc]
    
    # If neither condition is met, return the actual temperature
    mask_normal = ~(mask_hi | mask_wc)
    feels_like[mask_normal] = temp[mask_normal]
    
    return feels_like
df['feels_like'] = compute_feels_like(df['temperature_station1'], df['wind_speed'].apply(convert_wind_speed), df['humidity'])
df[['temperature_station1', 'humidity', 'wind_speed', 'feels_like']]

In [ ]:
# 6. Feature Engineering
def feature_engineering(X_train, y_train, X_val=None):
    temp_cols = ['temperature_station1', 'temperature_station2', 'temperature_station3', 'temperature_station4', 'temperature_station5', 'temperature_station6', 'temperature_station8', 'temperature_station9', 'temperature_station10']
    if X_val is not None:
        X_train, X_val = X_train.drop('temperature_station7', axis=1), X_val.drop('temperature_station7', axis=1)
        X_train['temperature_mean'], X_val['temperature_mean'] = X_train[temp_cols].mean(axis=1), X_val[temp_cols].mean(axis=1)
        # X_train, X_val = X_train.drop(columns=temp_cols), X_val.drop(columns=temp_cols)
        X_train, X_val = add_datetime_decomposition(X_train, datetime_col='date'), add_datetime_decomposition(X_val, datetime_col='date')
        X_train['feels_like'] = compute_feels_like(X_train['temperature_mean'], X_train['wind_speed'], X_train['humidity'])
        X_val['feels_like'] = compute_feels_like(X_val['temperature_mean'], X_val['wind_speed'], X_val['humidity'])
        return X_train.copy(), X_val.copy()
    else:
        X_train = X_train.drop('temperature_station7', axis=1)
        X_train['temperature_mean'] = X_train[temp_cols].mean(axis=1)
        # X_train= X_train.drop(columns=temp_cols)
        X_train = add_datetime_decomposition(X_train, datetime_col='date')
        X_train['feels_like'] = compute_feels_like(X_train['temperature_mean'], X_train['wind_speed'], X_train['humidity'])
        return X_train.copy()
    


In [ ]:
# Prepare X and y
X = df_train.copy().drop(columns=['electricity_demand'], axis=1)
y = df_train.copy().pop('electricity_demand')

# Run the evaluation
evaluate_pipeline(X, y)

#### Feature selection 

In [ ]:
# 7. Feature Selection and Dimensionality Reduction
def feature_selection(X_train, y_train, X_val=None):
    selected_columns = [
        'weather_condition', 'humidity', 'wind_speed', 'oil_brent_price_indicator', 'temperature_mean', 
        'feels_like', 'year', 'month', 'day', 'hour', 'dayofweek', 'is_weekend'
                        ]
    # selected_columns = X_train.drop('date', axis=1).columns
    if X_val is not None:
        return X_train[selected_columns], X_val[selected_columns]
    else:
        return X_train[selected_columns]

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.feature_selection import VarianceThreshold
import numpy as np
import pandas as pd

def feature_selection(X_train: pd.DataFrame, y_train: pd.Series, X_val: pd.DataFrame=None):
    """
    Sélection univariée de features adaptée à la régression.
    - Fit sur X_train/y_train
    - Transform sur X_train (et X_val si fourni)
    - Retourne (X_train_sel, X_val_sel) ou X_train_sel si X_val est None
    """
    
    selected_columns = [
        'weather_condition', 'humidity', 'wind_speed', 'oil_brent_price_indicator', 'temperature_mean', 
        'feels_like', 'year', 'month', 'day', 'hour', 'dayofweek', 'is_weekend'
                        ]
    X_train = X_train[selected_columns]
    if X_val is not None:
        X_val = X_val[selected_columns]
    
    # Select score
    p = X_train.shape[1]
    score_func = f_regression  # alternatif : mutual_info_regression
    k = min(30, max(5, int(np.sqrt(p))))
    k = min(k, p)
    
    # Select best features
    selector = SelectKBest(score_func=score_func, k=k)
    selector.fit(X_train, y_train)
    support = selector.get_support(indices=True)
    selected_cols = X_train.columns[support]
    
    # Transform
    X_train_sel = X_train[selected_cols].copy()
    if X_val is not None:
        X_val_sel = X_val[selected_cols].copy()
    
    # Affiche les features choisies
    print(f"[feature_selection] p={p} -> k={k} | selected: {list(selected_cols)}")
    
    return (X_train_sel, X_val_sel) if X_val is not None else X_train_sel


In [ ]:
# Prepare X and y
X = df_train.copy().drop(columns=['electricity_demand'], axis=1)
y = df_train.copy().pop('electricity_demand')

# Run the evaluation
evaluate_pipeline(X, y)

### Generating Submission File

In [ ]:
# Train and submit your results

In [ ]:
# Prepare X_train and y_train from your data
df_train =  pd.read_csv("module5_exercise_train.csv", sep=",")

X_train = df_train.drop(columns=['electricity_demand'], axis=1)
y_train = df_train['electricity_demand']

X_test =  pd.read_csv("module5_exercise_test.csv", sep=",")

In [ ]:
def train_and_predict_to_submit(X_train, y_train, X_test):
    model = LinearRegression()
    
    ### Sort datasets by date
    X_train = X_train.copy().sort_values('date')
    y_train = y_train.copy().reindex(X_train.index).reset_index(drop=True)
    X_train = X_train.reset_index(drop=True)
    X_test = X_test.copy().sort_values('date').reset_index(drop=True)

    ### Deal with absurd data (electricity demand)
    q_low, q_high = y_train.quantile(0.01), y_train.quantile(0.99)
    mask = y_train.between(q_low, q_high)
    X_train, y_train = X_train[mask], y_train[mask]

    X_train, y_train, X_test = handle_inconsistencies(X_train, y_train, X_test)
    X_train, y_train, X_test = handle_duplicates(X_train, y_train, X_test)
    X_train, y_train, X_test = handle_missing_values(X_train, y_train, X_test)
    X_train, y_train, X_test = handle_categorical(X_train, y_train, X_test)
    X_train, y_train, X_test = handle_outliers(X_train, y_train, X_test)
    X_train, X_test = feature_engineering(X_train, y_train, X_test)
    X_train, X_test = feature_selection(X_train, y_train, X_test)
    X_train, X_test = normalize(X_train, y_train, X_test, scaler)

    # Train the model on the entire training set
    print(f"Training model on entire dataset of shape: {X_train.shape}")
    model.fit(X_train, y_train)
    
    # Predict on the test set
    print(f"Predicting on test dataset of shape: {X_test.shape}")
    y_test_pred = model.predict(X_test)
    
    return y_test_pred

In [ ]:
# Call serve_model to train and predict
y_test_pred = train_and_predict_to_submit(X_train, y_train, X_test)

In [ ]:
y_test_pred.shape

In [ ]:
# Generating Submission File
submission = pd.DataFrame({
    'date': X_test['date'],
    'electricity_demand': y_test_pred
})

# Save the submission file
submission.to_csv('submission.csv', index=False, sep=',')
print("Submission file saved as 'submission.csv'.")

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(X_test['date'], y_test_pred)
# plt.xticks(rotation=45)
plt.xticks(X_test['date'][::20], rotation=45)
# plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()